# Fiche de test de la class Pathfinder de astar.py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.constants import pi
from astar import *
%matplotlib inline

## Paramètre de la Map

In [ ]:
x_max=3000.0
y_max=2000.0
nb_x=60
nb_y=40

DistanceObstacleMin=200.0
DistanceObstacleMax=500.0

## Création de la Map

In [ ]:
Map=Pathfinder(x_max, y_max, nb_x, nb_y, DistanceObstacleMin, DistanceObstacleMax)

In [ ]:
Map.iObsMax_x

In [ ]:
%%timeit
Map=Pathfinder(x_max, y_max, nb_x, nb_y, DistanceObstacleMin, DistanceObstacleMax)
#Etude du temps de creation de la map

## Rajoute d'obstable sur la Map

In [ ]:
Map.addObstacle(700.0,1000.0)
Map.addObstacle(1500.0,1000.0)
Map.addObstacle(1700.0,1000.0)
Map.addObstacle(2000.0,1000.0)
Map.addObstacle(2300.0,1000.0)
Map.addObstacle(2600.0,1000.0)
Map.addObstacle(1600.0,1000.0)
Map.addObstacle(1400.0,1000.0)
Obs=Map.getTable()
plt.pcolor(Map.X,Map.Y,Obs.T)
Obs[np.where(Obs<DistanceObstacleMin)]=0

## PathFinding

### Départ - Arrivé

In [ ]:
x_start = 0.0#readFloat(ser,"XrR?")
y_start = 0.0#readFloat(ser,"YrR?")
ang_start=0
x_fin = 1500.0
y_fin = 2000.0-y_max/nb_y
VtsMax=50.0
VtsAng=1.0

### Calcul du plus court Chemin

In [ ]:
Map.clean()
NodeFin = Map.pathfinding(x_start,y_start,ang_start,x_fin,y_fin,VtsMax,VtsAng)
TableX=[]
TableY=[]
TableAng=[]
TableTpsTrajet=[]
TableObs=[]
NodeFin.getParcours(TableX,TableY,TableAng,TableTpsTrajet,TableObs)

In [ ]:
%%timeit
Map.clean()
NodeFin = Map.pathfinding(x_start,y_start,ang_start,x_fin,y_fin,VtsMax,VtsAng)
#Etude du temps de pathfinding

In [ ]:
#TableX

### Tracé

In [ ]:
plt.pcolor(Map.X,Map.Y,Obs.T)
plt.colorbar()
x_p=np.array(TableX)
y_p=np.array(TableY)
plt.plot(x_p, y_p)
plt.colorbar()
plt.ylabel('x')
plt.xlabel('y')
plt.grid(True)
plt.ylim((0,2000.0))
plt.xlim((0,3000.0))
plt.show()

### Identification des points anguleux pour la determination du trajet

In [ ]:
Vmax=500
wait=0

In [ ]:
DeltaAng=np.convolve(TableAng,[1,-1],mode="same")
PointAnguleux=np.where((DeltaAng>=pi/4.0)|(DeltaAng<=-pi/4.0))
PointAnguleux=PointAnguleux[0]
j=0
NbPt=len(PointAnguleux)*2
for i in PointAnguleux:
    String="PstCons %s 0 %.4s %s %s %s 4 %s 0.1 10.0 10.0 \n" % (NbPt, TableAng[i], TableX[i], TableY[i],Vmax, wait)
    #ser.write(String) # pour un envoie de la chaine de caratères sur par port COM
    print(String[:-1])
    j=j+1
    String="PstCons %s 0 %.4s %s %s %s 3 %s 0.05 100.0 100.0 \n" % (NbPt, TableAng[i], TableX[i], TableY[i],Vmax, wait)
    #ser.write(String) # pour un envoie de la chaine de caratères sur par port COM
    print(String[:-1])
    j=j+1
#ser.write("Restart \n") # Ordre de demarrage à la carte de propultion
#ser.close()

In [ ]:
DeltaAng=np.convolve(TableAng,[1,-1],mode="same")
PointAnguleux=np.where((DeltaAng>=pi/4.0)|(DeltaAng<=-pi/4.0))
PointAnguleux=PointAnguleux[0]
j=0

for i in PointAnguleux:
    String="TuRn %.4s %s %s %s \n" % (j, TableAng[i],Vmax,wait)
    #ser.write(String) # pour un envoie de la chaine de caratères sur par port COM
    print(String[:-1])
    j=j+1
    String="GoTo %s %s %s %s %s \n" % (j, TableX[i], TableY[i],Vmax,wait)
    #ser.write(String) # pour un envoie de la chaine de caratères sur par port COM
    print(String[:-1])
    j=j+1
String="TuRn %.4s %s %s %s \n" % (j, TableAng[i],Vmax,wait)
#ser.write(String) # pour un envoie de la chaine de caratères sur par port COM
print(String[:-1])
#ser.write("Restart \n") # Ordre de demarrage à la carte de propultion
#ser.close()

In [ ]:
DeltaAng=np.convolve(TableAng,[1,-1],mode="same")
PointAnguleux=np.where((DeltaAng>=pi/4.0)|(DeltaAng<=-pi/4.0))
j=0
for i in PointAnguleux[0]:
    String="GoTo %s %s %s %s %s \n" % (j, TableX[i], TableY[i],Vmax,wait)
    #ser.write(String)
    print(String[:-1])
    j=j+1
    String="TuRn %s %s %s %s \n" % (j, TableAng[i],Vmax,wait)
    #ser.write(String)
    print(String[:-1])
    j=j+1
#ser.write("Restart \n")
#ser.close()

## Séparation de map : une fixe et une mobile qui pourrait être alimenté pour les capteurs

### Suppression de tous les obstacles de la Map astar

In [ ]:
Map.dellObstacle() # Suppression des obstacles
Obs=Map.getTable() # récupération de la carte d'obsetacle pour vérification pour affichage
plt.pcolor(Map.X,Map.Y,Obs.T)
plt.colorbar()
Obs[np.where(Obs<DistanceObstacleMin)]=0

### Importation de la map d'obstacle Fixe

In [ ]:
from PIL.Image import *

In [ ]:
imag=open("MapObs.png")
ObsMapFixe=np.zeros((nb_x,nb_y))+DistanceObstacleMax
for iy_obs in range(nb_y):
    for ix_obs in range(nb_x):
        if imag.getpixel((ix_obs, iy_obs)) == (0,0,0):
            for j in range(-Map.iObsMax_y,Map.iObsMax_y):
                for i in range(-Map.iObsMax_x,Map.iObsMax_x):
                    iy=iy_obs+j
                    ix=ix_obs+i
                    if (0<=ix<Map.nb_x) & (0<=iy<Map.nb_y):
                        NewDist = sqrt((ix-ix_obs)**2*Map.delta_x**2+(iy-iy_obs)**2*Map.delta_y**2)
                        if NewDist < ObsMapFixe[ix,iy]:
                            ObsMapFixe[ix,iy] = NewDist
ObsMapFixe[np.where(ObsMapFixe<DistanceObstacleMin)]=0
plt.pcolor(ObsMapFixe.T)

In [ ]:
ObsMapFixe

### Création d'une carte d'obstable mobile, versatile

In [ ]:
ObsMapMob=np.zeros((nb_x,nb_y))+DistanceObstacleMax

x_obs=700.0
y_obs=1000.0



ix_obs = int (x_obs/Map.delta_x)
iy_obs = int (y_obs/Map.delta_y)
for j in range(-Map.iObsMax_y,Map.iObsMax_y):
    for i in range(-Map.iObsMax_x,Map.iObsMax_x):
        iy=iy_obs+j
        ix=ix_obs+i
        if (0<=ix<Map.nb_x) & (0<=iy<Map.nb_y):
            NewDist = sqrt((ix-ix_obs)**2*Map.delta_x**2+(iy-iy_obs)**2*Map.delta_y**2)
            if NewDist < ObsMapMob[ix,iy]:
                ObsMapMob[ix,iy] = NewDist
#ObsMapMob[np.where(ObsMapMob<DistanceObstacleMin)]=0
plt.pcolor(ObsMapMob.T)

### Fusion des deux cartes

In [ ]:
ObsMap=np.minimum(ObsMapMob,ObsMapFixe)
plt.pcolor(ObsMap.T)
plt.colorbar()

### Ajout de de la carte comme obstacles à la Map astar

In [ ]:
Map.setObstacle(ObsMap)
Obs=Map.getTable()
Obs[np.where(Obs<DistanceObstacleMin)]=0
plt.pcolor(Map.X,Map.Y,Obs.T)
plt.colorbar()

### PathFinding

#### Départ - Arrivé

In [ ]:
x_start = 500.0#readFloat(ser,"XrR?")
y_start = 1700.0#readFloat(ser,"YrR?")
ang_start=0
x_fin = 2700.0
y_fin = 1300.0-y_max/nb_y
VtsMax=50.0
VtsAng=.05

#### Calcul du plus court Chemin

In [ ]:
Map.clean()
NodeFin = Map.pathfinding(x_start,y_start,ang_start,x_fin,y_fin,VtsMax,VtsAng)
TableX=[]
TableY=[]
TableAng=[]
TableTpsTrajet=[]
TableObs=[]
NodeFin.getParcours(TableX,TableY,TableAng,TableTpsTrajet,TableObs)

In [ ]:
%%timeit
Map.clean()
NodeFin = Map.pathfinding(x_start,y_start,ang_start,x_fin,y_fin,VtsMax,VtsAng)
#Etude du temps de pathfinding

#### Tracé

In [ ]:
plt.pcolor(Map.X,Map.Y,Obs.T)
plt.colorbar()
x_p=np.array(TableX)
y_p=np.array(TableY)
plt.plot(x_p, y_p)
plt.ylabel('x')
plt.xlabel('y')
plt.grid(True)
plt.ylim((0,2000.0))
plt.xlim((0,3000.0))
plt.show()